# Get partial RDMs for each Participant: semantic and action RDMs

In [1]:
import rsatoolbox
import csv 
import xlsxwriter
import numpy as np
import pandas as pd
import time as tm
import nibabel as nb
import glob
from os.path import isfile

# Prepare the data for the toolbox

4 channels -> Masks : Participants : condition (action or semantic) : beta images

In [2]:
filepath_mask = glob.glob(r'C:\Users\Márcia Vaz\Documents\GitHub\Dissertation\03_fMRI experiment\04_Masks\*')
participants_beta_folders = glob.glob(r'C:\Users\Márcia Vaz\Documents\GitHub\Dissertation\03_fMRI experiment\02_beta images\*')
all_data = []
all_RDMs_Action = []
all_RDMs_Semantic = []
ra=0
rs=0
for count0,m in enumerate(filepath_mask):
    mask = nb.load(m)
    mask_data = mask.get_fdata()
    # read the participants folders one by one
    for count1,participant in enumerate(participants_beta_folders):
        # read all beta images for each participant
        participant_beta_images_Action = glob.glob(participant + '\Action\*')
        participant_beta_images_Semantic = glob.glob(participant + '\Semantic\*')
        beta_array_created = False
        for count,beta_image in enumerate(participant_beta_images_Action):
            beta_img = nb.load(beta_image)
            get_beta_data = beta_img.get_fdata()
            roi_data = get_beta_data[mask_data==1]
            roi_data_nonNan = roi_data[~np.isnan(roi_data)]
            if count == 0:
                all_data = np.zeros(shape=(len(participant_beta_images_Action),roi_data_nonNan.size))
            all_data[count] = roi_data_nonNan
        data_Action = rsatoolbox.data.Dataset(all_data)
        RDM1_Action = rsatoolbox.rdm.calc_rdm(data_Action,method='euclidean')
        all_RDMs_Action.append(RDM1_Action.dissimilarities)
        #rsatoolbox.vis.show_rdm(RDM1_Action)
        #print(RDM1_Action.dissimilarities)
        
        for count,beta_image in enumerate(participant_beta_images_Semantic):
            beta_img = nb.load(beta_image)
            get_beta_data = beta_img.get_fdata()
            roi_data = get_beta_data[mask_data==1]
            roi_data_nonNan = roi_data[~np.isnan(roi_data)]
            
            if count == 0:
                all_data = np.zeros(shape=(len(participant_beta_images_Semantic),roi_data_nonNan.size))
            all_data[count] = roi_data_nonNan
        
        data_Semantic = rsatoolbox.data.Dataset(all_data)
        RDM1_Semantic = rsatoolbox.rdm.calc_rdm(data_Semantic,method='euclidean')
        all_RDMs_Semantic.append(RDM1_Semantic.dissimilarities)
        #rsatoolbox.vis.show_rdm(RDM1_Semantic)
    
#print(all_RDMs_Action)

In [3]:
excel_data = xlsxwriter.Workbook("RDMs_Action_data.xlsx")
worksheet = excel_data.add_worksheet()
for row, RDM in enumerate(all_RDMs_Action):
    for column, value in enumerate(RDM[0]):
        worksheet.write(row, column, value)
excel_data.close()
        

In [4]:
excel_data = xlsxwriter.Workbook("RDMs_Semantic_data.xlsx")
worksheet = excel_data.add_worksheet()
for row, RDM in enumerate(all_RDMs_Semantic):
    for column, value in enumerate(RDM[0]):
        worksheet.write(row, column, value)
excel_data.close()
        